In [1]:
# (C) Copyright Aaron Goldberg, 2022.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

In [20]:
import strawberryfields as sf
from strawberryfields.ops import Sgate, Rgate, BSgate, MeasureFock
from strawberryfields.tdm import borealis_gbs, full_compile, get_mode_indices
import numpy as np

In [21]:
# Compilation will make sure the specs match an actual device, so must be done using a particular device and its characteristics
eng = sf.RemoteEngine("borealis")
device = eng.device

In [22]:
# Let's define our circuit manually
modes = 2

# squeezing-gate parameters. These will be rounded to the closest one supported by hardware, so that can by verified
#r0 = device.certificate["squeezing_parameters_mean"]["high"]
r = [1.2] * modes

# rotation-gate parameters
phi_0 = [0.] * modes
phi_1 = [0.] * modes
phi_2 = [0.] * modes
phi_0[1] = -np.pi/2 # Makes sure we have a 50:50 BS instead of a symmetric one

# beamsplitter parameters. Set irrelevant transmission parameters to 0 to bypass all nonessential loops and mitigate losses
T_0 = [0.] * modes
T_1 = [0.] * modes
T_2 = [0.] * modes
T_0[1] = 0.5 # 50:50 BS
alpha_0 = np.arccos(np.sqrt(T_0))
alpha_1 = np.arccos(np.sqrt(T_1))
alpha_2 = np.arccos(np.sqrt(T_2))

# the travel time per delay line in time bins
delay_0, delay_1, delay_2 = 1, 6, 36

# set the first beamsplitter arguments to 'T=1' ('alpha=0') to fill the
# loops with pulses
alpha_0[:delay_0] = 0.0
alpha_1[:delay_1] = 0.0
alpha_2[:delay_2] = 0.0

# The gate arguments need to be defined as lists, so if they were defined with numpy we need to cast them to lists
gate_args = {
    "Sgate": r,
    "loops": {
        0: {"Rgate": phi_0, "BSgate": alpha_0.tolist()},
        1: {"Rgate": phi_1, "BSgate": alpha_1.tolist()},
        2: {"Rgate": phi_2, "BSgate": alpha_2.tolist()},
    },
}

In [23]:
# Now compile:
gate_args_list = full_compile(gate_args, device)

In [24]:
# Verify components if desired
#gate_args_list[2]

In [25]:
delays = [1, 6, 36]
vac_modes = sum(delays)

n, N = get_mode_indices(delays)

In [26]:
# Use this if want to manually add realistic losses gleaned from the device characteristics, can then be tweaked
eta_glob = device.certificate["common_efficiency"]
etas_loop = device.certificate["loop_efficiencies"]
etas_ch_rel = device.certificate["relative_channel_efficiencies"]

prog_length = len(gate_args_list[0])
reps = int(np.ceil(prog_length / 16))
etas_ch_rel = np.tile(etas_ch_rel, reps)[:prog_length]
etas_ch_rel = etas_ch_rel.tolist()

gate_args_list += [etas_ch_rel]

from strawberryfields.ops import LossChannel
print("minimum loss for any mode is: ", 1-eta_glob)

minimum loss for any mode is:  0.614


In [27]:
prog = sf.TDMProgram(N)

with prog.context(*gate_args_list) as (p, q):
    Sgate(p[0]) | q[n[0]]
    #LossChannel(eta_glob) | q[n[0]]
    for i in range(len(delays)):
        Rgate(p[2 * i + 1]) | q[n[i]]
        BSgate(p[2 * i + 2], np.pi / 2) | (q[n[i + 1]], q[n[i]]) # Get rid of phases to make regular 50:50 BS instead of symmetric BS
        #LossChannel(etas_loop[i]) | q[n[i]]
    #LossChannel(p[7]) | q[0]
    MeasureFock() | q[0]

In [29]:
# Let's check the transfer matrix for the relevant modes
prog_passive = sf.TDMProgram(N)
with prog_passive.context(*gate_args_list) as (p, q):
    for i in range(len(delays)):
        Rgate(p[2 * i + 1]) | q[n[i]]
        BSgate(p[2 * i + 2], np.pi / 2) | (q[n[i + 1]], q[n[i]])
prog_passive.space_unroll()
prog_passive = prog_passive.compile(compiler="passive")
transfer_matrix_tot = prog_passive.circuit[0].op.p[0]
# crop out the vacuum modes
transfer_matrix = transfer_matrix_tot[vac_modes:prog_length, vac_modes:prog_length]
print("The transfer matrix is: ", transfer_matrix.round(15))

The transfer matrix is:  [[-0.70710678+0.j         -0.70710678+0.j        ]
 [-0.        -0.70710678j  0.        +0.70710678j]]


In [30]:
compile_options = {
    "device": device,
    "realistic_loss": False,
}
run_options = {
    "shots": 0,
    "crop": True,
    "space_unroll": True,
}

eng_sim = sf.Engine(backend="gaussian")
results_sim = eng_sim.run(prog, **run_options, compile_options=compile_options) # Using automated loss
#results_sim = eng_sim.run(prog, shots=None, space_unroll=True, crop=True) # Using manual loss from specific device

In [31]:
state = results_sim.state
print("Number of modes: ",state.num_modes)
high_cutoff=40
low_cutoff=40
mode0=state.reduced_dm(0,cutoff=high_cutoff) #This does some weird things when the cutoff is too small!
print("trace, purity of mode 0 calculated with Fock (faulty!): ",np.trace(mode0),np.trace(np.matmul(mode0,mode0)))
mode1=state.reduced_dm(1,cutoff=high_cutoff) #This does some weird things when the cutoff is too small!
print("trace, purity of mode 1 calculated with Fock (faulty!): ",np.trace(mode1),np.trace(np.matmul(mode1,mode1)))
gauss0=state.reduced_gaussian(0)
gauss1=state.reduced_gaussian(1)

# If I make a state using the covariance matrix and means listed above, I get the wrong answer
from thewalrus.quantum import state_vector, density_matrix
mu0=gauss0[0]
cov0=gauss0[1]
#psi0 = state_vector(mu0, cov0, normalize=False, cutoff=low_cutoff) # Can only use this for pure states
rho0 = density_matrix(mu0, cov0, normalize=False, cutoff=low_cutoff)
#print(psi0) # Can only use this for pure states
#print(np.diag(rho0))
#print(np.diag(mode0))
mu1=gauss1[0]
cov1=gauss1[1]
#psi1 = state_vector(mu1, cov1, normalize=False, cutoff=low_cutoff) # Can only use this for pure states
rho1 = density_matrix(mu1, cov1, normalize=False, cutoff=low_cutoff)
#print(psi1) # Can only use this for pure states
#print(rho1)
#print(mode1)
print("Quadrature coherence scale from mode 0:",1.+2*np.sum([n*((-1)**n) * rho0[n][n] for n in range(low_cutoff)])/np.sum([((-1)**n) * rho0[n][n] for n in range(low_cutoff)]))
print("Quadrature coherence scale from mode 1:",1.+2*np.sum([n*((-1)**n) * rho1[n][n] for n in range(low_cutoff)])/np.sum([((-1)**n) * rho1[n][n] for n in range(low_cutoff)]))
print("If it was pure we'd have QCS of 1+2 nbar:",1.+2.*np.sinh(gate_args_list[0][0])**2)
print("nbar",np.sum([n * rho1[n][n] for n in range(low_cutoff)]))
print("purity of initial state from output Fock",np.sum([((-1)**n) * rho1[n][n] for n in range(low_cutoff)]))
print("g^2(0) mode 0:",np.sum([n*(n-1)*rho0[n][n] for n in range(low_cutoff)])/(np.sum([n * rho0[n][n] for n in range(low_cutoff)])**2))
print("g^2(0) mode 1:",np.sum([n*n*rho1[n][n] for n in range(low_cutoff)])/(np.sum([n * rho1[n][n] for n in range(low_cutoff)])**2)-1/(np.sum([n * rho0[n][n] for n in range(low_cutoff)])))

Number of modes:  2
trace, purity of mode 0 calculated with Fock (faulty!):  (0.9999999999999996+0j) (0.9999999999999987-2.8846906600061224e-18j)
trace, purity of mode 1 calculated with Fock (faulty!):  (1.0000000000000002+0j) (0.9999999999999998+5.986719008667593e-19j)
Quadrature coherence scale from mode 0: (5.061068610706544+1.3370574463074536e-14j)
Quadrature coherence scale from mode 1: (5.061068610706653-4.3859562589197643e-16j)
If it was pure we'd have QCS of 1+2 nbar: 5.0669332680766805
nbar (2.030391840554952+3.3173941741084467e-16j)
purity of initial state from output Fock (0.9999298387631703+1.020140322132278e-17j)
g^2(0) mode 0: (3.4700314113413-9.097739264057594e-17j)
g^2(0) mode 1: (3.4700314113413016-1.6781400695817703e-16j)


In [32]:
# np.savetxt('sample_data.out',mode0, delimiter=',')

In [33]:
print(rho1.round(14))

[[ 5.74156636e-01+0.j          0.00000000e+00+0.j
  -2.16264798e-01+0.25243013j ...  0.00000000e+00+0.j
   3.58901809e-03-0.00288912j  0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j         -0.00000000e+00+0.j
   0.00000000e+00+0.j         ...  0.00000000e+00+0.j
   0.00000000e+00+0.j         -0.00000000e+00+0.j        ]
 [-2.16264798e-01-0.25243013j  0.00000000e+00+0.j
   1.92441273e-01+0.j         ...  0.00000000e+00+0.j
  -2.62207240e-03-0.00048969j  0.00000000e+00+0.j        ]
 ...
 [ 0.00000000e+00+0.j          0.00000000e+00-0.j
   0.00000000e+00+0.j         ... -0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00-0.j        ]
 [ 3.58901809e-03+0.00288912j  0.00000000e+00+0.j
  -2.62207240e-03+0.00048969j ...  0.00000000e+00+0.j
   3.69726449e-05+0.j          0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j         -0.00000000e+00-0.j
   0.00000000e+00+0.j         ...  0.00000000e+00+0.j
   0.00000000e+00+0.j         -0.00000000e+00+0.j        ]]
